# TPSC and the Mermin-Wagner theorem

## Introduction

The Mermin-Wagner theorem states that a continuous-symmetry cannot be broken at finite temperature in two dimensions. So, antiferromagnetism is impossible in two dimensions, contrary to the prediction of RPA. Here is a heuristic proof. Assume that the spins are collinear in the $z$ direction. Then the free energy density contains a term proportional to $(\nabla S^z)^2/2$. In Fourier space, this becomes $-q^2|S^z(\mathbf{q})|^2/2$. In the long wavelength limit, where fluctuations are slow, we can use the classical equipartition theorem so that 

$$|S^z(\mathbf{q})|^2=\frac{k_BT}{q^2}$$. 

This means that the local moment diverges because of an infrared divergence

$$\left<(S^z)^2\right>\sim \int d^2q |S^z(\mathbf{q})|^2\sim \int d^2q\frac{k_BT}{q^2}=\infty$$. 

We come to an absurdity, which proves that the hypothesis is wrong, there is no long-range order. 

To see that TPSC satisfies the Mermin-Wagner theorem, first note that the spin susceptibility has the following spectral representation

$$\chi_{sp}(\mathbf{q},i\omega_n)=\int\frac{d\omega}{\pi}\frac{\chi_{sp}''(\mathbf{q},\omega)}{\omega-i\omega_n}=\int\frac{d\omega}{\pi}\frac{\chi_{sp}''(\mathbf{q},\omega)\omega}{\omega^2+(\omega_n)^2}.$$

The last equality follows from the fact $\chi_{sp}''(\mathbf{q},\omega)$ is odd in frequency. This last result shows that the finite Matsubara frequencies should be regular. The largest contribution is the zero-Matsubara frequency one. This allows us to give a rough idea of why the theorem is satisfied by focusing on the zero Matsubara frequency contribution. 

Let us then write the self-consistency condition for $U_{sp}$ as follows :

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q}} \frac{\chi_0(\mathbf{q},0)}{1-\frac{U_{sp}}{2}\chi_0(\mathbf{q},0)}=n-2\left< n_\uparrow n_\downarrow\right>-C(T)
\end{equation}

where $C(T)$ contains the non-singular contribution of the finite Matsubara frequencies. 

Calling the right-hand side $C'(T)$, expanding the denominator around the maximum at $\mathbf{Q}=(\pi,\pi)$ and shifting the origin of the wave vector integration to $\mathbf{Q}=(\pi,\pi)$, the self-consistency condition can be written as  

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q}} \frac{A}{\xi^{-2}+q^2}=C'(T)
\end{equation}
where $A$ is a constant and $\xi$ the correlation length contains the value of $U_{sp}$. Since the right-hand side is finite, $\xi$ adjusts itself not to become infinite, otherwise the left-hand side diverges. The divergence of the susceptibility can occur only at $T=0$ where we cannot treat the non-zero Matsubara frequencies separately. 




## Code from previous notebooks

To study the temperature dependence we will reuse the code from the previous TPSC notebooks. Please look through the functions and make sure that they are familiar.

In [ ]:
# Imports 
from triqs.lattice import BravaisLattice, BrillouinZone
from triqs.gf import MeshBrZone, MeshImFreq, Gf, MeshProduct, inverse, Idx
from h5 import HDFArchive
from triqs.plot.mpl_interface import plt, oplot
plt.rcParams["figure.dpi"] = 100 # make figures bigger
%matplotlib inline
import numpy as np
from scipy.optimize import fsolve, brentq
from math import cos, pi

Here is the `c++` accelerated Lindhard bubble calculator.

In [ ]:
%reload_ext cpp2py.magic

In [ ]:
%%cpp2py -C triqs --cxxflags '-std=c++20'
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

// The type of a Green function : (k,omega) -> Complex number
using g_k_w_vt = gf_view<prod<brzone, imfreq>, scalar_valued>;
using g_r_t_t  = gf<prod<cyclat, imtime>, scalar_valued>;

auto bubble(g_k_w_vt G0) {
    
    // Fourier Transformation of k, \omega to obtain g(r,t)
    auto grt = make_gf_from_fourier<0,1>(G0);
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mr, mt] = grt.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;
    
    // A new mesh for chi, with a bosonic statistics, but same size as mt.
    auto mtb = gf_mesh<imtime>{beta, Boson, mt.size()};
    
    // Build chi (r, tau) with this new mesh.
    auto chi0 = g_r_t_t{{mr, mtb}};

    // we fill chi : chi(r, tau) = 2 * g(-r, beta - tau) * g(r, tau)
    for (auto const &r : mr)
        for (auto const &t : mtb)
            chi0[r, t] = 2*grt(-r, beta - t) * grt(r, t);

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(chi0);
}

## Fast $\chi_0$ calculator

Since we have to recompute the Lindhard bubble $\chi_0$ for every temperature we provide an implementation of the bubble calculation below (including a faster calculator for the lattice Green's function $G_0(\mathbf{k}, i\omega)$).

In [ ]:
eps = np.vectorize(lambda k: -2*t*(np.cos(k[0]) + np.cos(k[1])), signature='(d)->()')

def get_chi0(beta):
    # Create and Init G0
    iwmesh = MeshImFreq(beta=beta, S='Fermion', n_iw=n_iw)
    iwarr = np.array(list(iwmesh.values()))
    karr = np.array(list(kmesh.values()))
    G0 = Gf(mesh = MeshProduct(kmesh, iwmesh), target_shape = [])
    G0.data[:,:] = 1.0 / (iwarr[None,:] - eps(karr)[:,None])
    
    return bubble(G0)

Here are the functions used for the TPSC self consistency.

In [ ]:
def chi_rpa(chi0, U):
    """Compute chi_rpa from chi"""
    return chi0 * inverse(1 - 0.5 * U * chi0)

def trace_chi(chi):
    """Given chi_kw, it computes sum_k sum_\nu chi(k,\nu)""" 
    kmesh, wmesh = chi.mesh.components
    trace = chi.data.sum() / len(kmesh) / wmesh.beta
    return trace.real

def Usp_root(Usp, chi0, n, U):
    """Sets the self-consistency for U_sp as the problem of finding roots"""
    tr_chi_sp = trace_chi(chi_rpa(chi0, U=Usp))
    diff = tr_chi_sp + 0.5 * Usp/U * n**2 - n
    return diff

def Uch_root(Uch, chi0, n, U, docc):
    tr_chi = trace_chi(chi_rpa(chi0, U=-Uch))
    diff = tr_chi - 2 * docc - n + n**2
    return diff

def solve_tpsc(chi0, U, n):
    Uc = 2/chi0([np.pi,np.pi,0],0).real
    Usp = brentq(Usp_root, 0, Uc-1e-6, args=(chi0, n, U), xtol=1e-2)
    docc = 0.25 * Usp / U * n**2
    Uch = brentq(Uch_root, 0, 100, args=(chi0, n, U, docc), xtol=1e-2)
    return Usp, Uch, docc, Uc

## <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise: Temperature sweep

The goal of the exercise is to compute a temperature sweep for $n=1$ and $U=4$ and examine the validity of the Mermin-Wagner theorem in the TPSC approximation.

We compute the spin structure factor which is given by 
$$S(\mathbf{q})\equiv T\sum_n \chi_{sp}(\mathbf{q},i\omega_n)$$

We want to reproduce the following figure from the
__[paper](https://jp1.journaldephysique.org/articles/jp1/abs/1997/11/jp1v7p1309/jp1v7p1309.html)__ :
<img src="./imgs/Fig7.png" alt="Drawing" style="width: 250px;"/>


## Parameters

Here are suggested discretization parameters for the $n=1$ and $U=4$ temperature sweep.

In [ ]:
n = 1.0
U = 4.0
t = 1.0
n_k = 32 # Number of k points per direction
n_iw = 128 # Number of Matsubara frequencies

# Two unit vectors in R3
BL = BravaisLattice([(1, 0, 0), (0, 1, 0)])
BZ = BrillouinZone(BL)
kmesh = MeshBrZone(BZ, n_k=n_k)

## RPA spin structure factor $S_{RPA}$ as a function of temperature $T$

For comparison we compute the RPA spin structure factor $S_{RPA}$ for a range of temperatures. Note that $T_c^{(RPA)} \approx 0.75$.

In [ ]:
T_rpa_vec = np.concatenate((np.arange(4., 3., -1.), np.arange(3, 0.75, -0.2)))
S_rpa_vec = np.zeros_like(T_rpa_vec)

print(''.join('| %-11s' % s for s in ['T', 'beta', 'S_rpa']), '|')
print('-'*41)

for idx, T in enumerate(T_rpa_vec):

    beta = 1. / T
    chi0_qw = get_chi0(beta)
    chi = chi_rpa(chi0_qw, U)
    
    S_rpa = chi[Idx(n_k//2, n_k//2, 0), :].data.sum().real 
    S_rpa_vec[idx] = S_rpa
    
    print(''.join('| %4.4E ' % x for x in [T, beta, S_rpa]), '|')

## TPSC spin structure factor $S_{TPSC}$ as a function of temperature

Using the ansatz $U_{sp}\left<n_\uparrow\right> \left<n_\downarrow\right>=U\left<n_\uparrow n_\downarrow\right>$, the spin susceptibility obeys

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},i\omega_n} \frac{\chi_0(\mathbf{q},i\omega_n)}{1-\frac{U\left<n_\uparrow n_\downarrow\right>}{2\left<n_\uparrow\right> \left<n_\downarrow\right>}\chi_0(\mathbf{q},i\omega_n)}=n-2\left< n_\uparrow n_\downarrow\right>
\end{equation}

When the susceptibility increases, $\left<n_\uparrow n_\downarrow\right>$ on the right-hand side decreases, but then the denominator of the spin susceptibility will lead to a decrease in susceptibility.

More rigorously, we can see that dimension is important here. Let us repeat the argument at the beginning of the notebook. The right-hand side of the equation cannot diverge. Also, on the left-hand side, note that the most divergent contribution is the zero Matsubara frequency, as one can see from the spectral representation and $\chi''(\mathbf{q},\omega)=-\chi''(\mathbf{q},-\omega)$

\begin{equation}
\chi(\mathbf{q},i\omega_n)=\int \frac{d\omega}{\pi}\frac{\chi''(\mathbf{q},\omega)}{\omega-i\omega_n}=\int \frac{d\omega}{\pi}\frac{\omega\chi''(\mathbf{q},\omega)}{\omega^2+\omega_n^2}.
\end{equation}

Using these results, the non-singular finite Matsubara frequency terms can be put on the right-hand side of the sum rule and all that is left is 

\begin{equation}
T\int d^2q \frac{a}{\xi^{2}+q^2}\sim C'(T)
\end{equation}

where we have expanded the susceptibility around $(\pi,\pi)$, gone from sum to integral and shifted the origin of integration so that now $\mathbf{q}$ is the deviation from $(\pi,\pi)$. On dimensional grounds, the left-hand side is logarithmic in two dimensions so that the correlation length scales like $\exp(C'(T)/T)$.


## <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise: Spin structure factor

Compute the TPSC spin structure factor for a range of temperatures $T \in [0.1, 4]$ and plot $S_{TPSC}$ and $S_{RPA}$ and determine whether the Mermin-Wagner theorem holds.

## <i class="fa fa-gear fa-x" style="color: #186391"></i> Excercise: Critical temperature and double occupancy

To see the divergencies it is useful to also study the inverse spin structure factor $S^{-1}$. Plot $S^{-1}$ and see at what temperatures the curves intercept $S^{-1}=0$ to determine the critical temperatures $T_c$ of RPA and TPSC.

Q: Plot the double occupancy and explain its behaviour as a function of temperature. What is the $T \rightarrow \infty$ limit of the double occupancy?

## Question

There seems to be a finite transition temperature even for TPSC.

If there is a numerical problem leading to that, can you identify it?